In [1]:
import pandas as pd

df = pd.read_excel('Time Series.xlsx')

print(df.shape)
df.head(2)

(10885, 4)


,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection


In [2]:
df.Name.unique()

array(['Priyanka', 'Jyoti', 'Deepti', 'Sharan', 'Ravi'], dtype=object)

In [3]:
# df['Start'] = pd.to_datetime(df['Start'])
# df['End'] = pd.to_datetime(df['End'])

In [4]:
df.dtypes

Name                object
Start       datetime64[ns]
End         datetime64[ns]
Activity            object
dtype: object

- Above is date-time object with nano-second resolution

In [5]:
df.head()

,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection
2,Jyoti,2023-06-08 08:19:52.620,2023-04-08 05:55:52.620,Updates
3,Priyanka,2023-09-21 15:31:52.620,2023-05-27 10:43:52.620,Reporting
4,Priyanka,2023-10-07 03:31:52.620,2023-04-30 13:07:52.620,Reply to Customers


- **`itertools`**: A Python module that provides fast, memory-efficient tools to create and manipulate iterators for looping and data aggregation.
- **`itertools.groupby`**: A function that groups consecutive elements of an iterable based on a key function, returning key-value pairs where each key is associated with its corresponding group.

In [6]:
df.iloc[0][['Start', 'End']].values

array([Timestamp('2023-10-10 22:43:52.620000'),
       Timestamp('2023-07-20 03:31:52.620000')], dtype=object)

In [7]:
df.iloc[0][['Start', 'End']].values[0]

Timestamp('2023-10-10 22:43:52.620000')

#### Merge Overlapping or Continuous Periods:

In [8]:
i=1
temp_df=0

for group in df.groupby('Name'):
    temp_df = group
    i+=1
    if i==1:
        break

In [9]:
type(temp_df)

tuple

In [10]:
print(temp_df[0])
temp_df[1].head()

Sharan


,Name,Start,End,Activity
9,Sharan,2023-06-08 13:07:52.620,2023-09-24 01:07:52.620,Reply to Customers
14,Sharan,2023-09-18 10:43:52.620,2023-07-31 03:31:52.620,Business Development
17,Sharan,2023-09-06 08:19:52.620,2023-05-11 15:31:52.620,Inspection
19,Sharan,2023-09-06 01:07:52.620,2023-07-17 20:19:52.620,Podcast
21,Sharan,2023-10-09 08:19:52.620,2023-08-01 13:07:52.620,Reply to Customers


In [11]:
from itertools import groupby

def merge_periods(group):
    merged_periods = {}
    current_period = group.iloc[0][['Start', 'End']].values
    current_activities = [group.iloc[0]['Activity']]
    
    for i in range(1, len(group)):
        row = group.iloc[i]
        if row['Start'] <= current_period[1]:  # If overlapping or continuous
            current_period[1] = max(current_period[1], row['End'])  # Extend the end time
            current_activities.append(row['Activity'])  # Aggregate activity
        else:
            merged_periods[(current_period[0], current_period[1])] = current_activities
            current_period = row[['Start', 'End']].values
            current_activities = [row['Activity']]
    
    merged_periods[(current_period[0], current_period[1])] = current_activities
    return merged_periods


In [12]:
bot_activity = {}

for group in df.groupby('Name'):
    bot_activity[group[0]] = merge_periods(group[1])

In [28]:
i=0
for key in bot_activity['Deepti'].keys():
    print(key[0])
    if i==0:
        break
# [str((Timestamp('2023-03-29 15:31:52.620000'), Timestamp('2023-10-15 15:31:52.620000')))]

2023-08-02 13:07:52.620000


In [29]:
bot_activity['Deepti']['2023-08-02 13:07:52.620000']

KeyError: '2023-08-02 13:07:52.620000'